In [ ]:
# ALL INPUTS: (start="MM/DD/YYYY",end="MM/DD/YYYY")
# conference() <- bills considered through conference
# unrep_house() & unrep_senate() <- bills considered but NOT reported
# amendments_senate() <- senate amendments considered
# rep_senate() & rep_house() <- number of bills reported by committee
# cloture()
# workingdays()

In [12]:
def conference(start='01/01/2020',end = None):
    from PyPDF2 import PdfFileReader
    import PyPDF2
    import io
    import requests 
    from datetime import datetime, timedelta
    import pandas as pd
    import re
    if end == None:
        dateend = end = datetime.now().date()       
    else:
        dateend = datetime.strptime(end, '%m/%d/%Y').date()
    datestart = datetime.strptime(start, '%m/%d/%Y').date()
    diff = (dateend - datestart).days
    url = ("https://www.govinfo.gov/link/ccal/house/Bills%20Through%20Conference?link-type=pdf&publishdate=" + str(dateend.strftime("%Y")) + "-" + str(dateend.strftime("%m")) + "-" + str(dateend.strftime("%d")))
    for i in range(diff):
        if '200' in str(requests.get(url)):
            break
        else:
            dateend = dateend -  timedelta(days=1)
            url = ("https://www.govinfo.gov/link/ccal/house/Bills%20Through%20Conference?link-type=pdf&publishdate=" + str(dateend.strftime("%Y")) + "-" + str(dateend.strftime("%m")) + "-" + str(dateend.strftime("%d")))
    r = requests.get(url)
    f = io.BytesIO(r.content)
    reader = PyPDF2.PdfFileReader(f)
    page_num = 0
    for i in range(10):
        try:
            contents = reader.getPage(page_num).extractText().split('\n')
        except:
            break
        else:
            page_num = page_num + 1
    l = ["H.R.","S.","H.Con","H.J","S.Con","S.J","S.Con"]
    res = []
    for x in range(page_num):
        contents = reader.getPage(x).extractText().split('\n')
        for row in contents:
            t = str(row)
            for i in l:
                if i in t:
                    val = t.find(i)
                    val2 = t.find("Š")
                    if t[val:val2] in res:
                        continue
                    try:
                        type(int(re.sub("[^0-9]", "", t[val:val2])))
                    except:
                        continue
                    else:
                        res.append(t[val:val2])
    l = [[len(res),res,url]]
    ret = pd.DataFrame(l,columns = [str("Measures Considered through Conference "+start+" and "+dateend.strftime("%m/%d/%Y")+"."),"Measures","Check Here"])
    return(ret)

In [96]:
def reportedbills(start,end):
    from PyPDF2 import PdfFileReader
    import PyPDF2
    import io
    import requests 
    from datetime import datetime, timedelta
    import pandas as pd
    import re
    from lxml import html
    import calendar 
    from dateutil.relativedelta import relativedelta
    if end == None:
        dateend = end = datetime.now().date()       
    else:
        dateend = datetime.strptime(end, '%m/%d/%Y').date()
    datestart = datetime.strptime(start, '%m/%d/%Y').date()

    def diff_month(d2,d1):
        return (d1.year - d2.year) * 12 + d1.month - d2.month
    nummonths = int(diff_month(datestart,dateend))

    results = pd.DataFrame(columns=["Senate","House"])

    for i in range(nummonths+1):
        date2 = datestart.replace(day=int(calendar.monthrange(int(datestart.strftime("%Y")),int(datestart.strftime("%m")))[1]))
        url = "https://www.congress.gov/advanced-search/command-line?query=legislativeSource:%22Congressional%20Record%22%20title:%22*Resume*%22%20crArticleDateStr:[%22"+str(datestart.strftime("%Y-%m-%d"))+"%22%20TO%20%22"+str(date2.strftime("%Y-%m-%d"))+"%22]&searchResultViewType=expanded&KWICView=false"
        webpage = requests.get(url)
        tree = html.fromstring(webpage.content)
        try:
            l = tree.xpath("//div[@class='cdg-summary-wrapper']//ul//li/a/@href")[0]
        except:
            try:
                l = tree.xpath("//*//li[@class='expanded']//span[@class='result-heading congressional-record-heading']//@href")[0]
                webpage2 = requests.get(l)
                tree2 = html.fromstring(webpage2.content)
                f2 = io.BytesIO(webpage2.content)
                l2 = tree2.xpath("//div[@class='cdg-summary-wrapper']//ul//li/a/@href")[0]
                l = l2
            except:
                datestart += relativedelta(months=1)
                date2 = datestart.replace(day=int(calendar.monthrange(int(datestart.strftime("%Y")),int(datestart.strftime("%m")))[1]))
                url = "https://www.congress.gov/advanced-search/command-line?query=legislativeSource:%22Congressional%20Record%22%20title:%22*Resume*%22%20crArticleDateStr:[%22"+str(datestart.strftime("%Y-%m-%d"))+"%22%20TO%20%22"+str(date2.strftime("%Y-%m-%d"))+"%22]&searchResultViewType=expanded&KWICView=false"
                webpage = requests.get(url)
                tree = html.fromstring(webpage.content)
                try:
                    l = tree.xpath("//div[@class='cdg-summary-wrapper']//ul//li/a/@href")[0]
                except:
                    try:
                        l = tree.xpath("//*//li[@class='expanded']//span[@class='result-heading congressional-record-heading']//@href")[0]
                        webpage2 = requests.get(l)
                        tree2 = html.fromstring(webpage2.content)
                        f2 = io.BytesIO(webpage2.content)
                        l2 = tree2.xpath("//div[@class='cdg-summary-wrapper']//ul//li/a/@href")[0]
                        l = l2
                    except:
                        break
        url = 'https://www.congress.gov'+l
        webpage = requests.get(url)
        f = io.BytesIO(webpage.content)
        reader = PyPDF2.PdfFileReader(f)
        txt = reader.getPage(0).extractText().split('\n')
        vals = str(txt)[str(txt).find("through")-30:str(txt).find("through")+30]
        try:
            dates = list()
            for match in re.finditer(r'(\b\d{1,2}\D{0,3})?\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(Nov|Dec)(?:ember)?)\D?(\d{1,2}\D?)?\D?((19[7-9]\d|20\d{2})|\d{2})', vals,re.S):
                dates.append(match.group())
            through = str(dates[0]) + " through " + str(dates[1])
        except:
            match1 = re.search(r'(\b\d{1,2}\D{0,3})?\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(Nov|Dec)(?:ember)?)\D?(\d{1,2}\D?)?\D?((19[7-9]\d|20\d{2})|\d{2})', vals)
            match = re.search(r'(\b\d{1,2}\D{0,3})?\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(Nov|Dec)(?:ember)?)\D?(\d{1,2})', vals)
            through = str(match.group()+" through "+match1.group())
        reported_str = str(txt)[str(txt).find("Measures reported"):str(txt).find("Measures reported")+str(txt)[str(txt).find("Measures reported"):].find("Senate bills")]
        reported_str = re.sub("\*","",reported_str)
        reported_str = re.sub("\."," ",reported_str)
        reported_str = re.sub(",","",reported_str)
        res = [int(i) for i in reported_str.split() if i.isdigit()]
        res = [[res[0],res[1],url]]
        res = pd.DataFrame(res,columns=["Senate","House","URL"],index=[("Number of Bills Reported in Range: "+through)])
        results = results.append(res)
        datestart += relativedelta(months=1)

    return(results)


In [13]:
def unrep_house(start = "01/01/2020",end=None):
    from lxml import html
    import requests
    import re
    import pandas as pd
    from datetime import datetime, timedelta
    if end == None:
        dateend = end = datetime.now().date()       
    else:
        dateend = datetime.strptime(end, '%m/%d/%Y').date()
    datestart = datetime.strptime(start, '%m/%d/%Y').date()
    webpage = requests.get("https://www.congress.gov/advanced-search/command-line?query=actionCode%3A7000+NOT+5000+latestActionDateStr%3A%5B%22"+str(datestart.strftime("%Y-%m-%d"))+"%22+TO+%22"+str(dateend.strftime("%Y-%m-%d"))+"%22%5D&searchResultViewType=compact&q=%7B%22type%22%3A%5B%22bills%22%2C%22joint-resolutions%22%5D%7D&pageSize=250")
    tree = html.fromstring(webpage.content)
    try:
        iterations = int(re.sub("[^0-9]", "", str(tree.xpath("//div[@class='pagination']//span[@class='results-number']/text()"))))
    except:
        iterations = 1
    results = []
    for i in range(int(iterations)):
        webpage = requests.get("https://www.congress.gov/advanced-search/command-line?query=actionCode%3A7000+NOT+5000+latestActionDateStr%3A%5B%22"+str(datestart.strftime("%Y-%m-%d"))+"%22+TO+%22"+str(dateend.strftime("%Y-%m-%d"))+"%22%5D&searchResultViewType=compact&q=%7B%22type%22%3A%5B%22bills%22%2C%22joint-resolutions%22%5D%7D&pageSize=250&page="+str(i))
        tree = html.fromstring(webpage.content)
        Bill_Code = tree.xpath("//*//ol[@class='basic-search-results-lists expanded-view']//li[@class='compact']/span[@class='result-heading']/a/text()")
        Latest = tree.xpath("//*//li[@class='compact']//span[@class='result-item'][./strong='Latest Action:']/span[1]/text()[1]")
        Congress = tree.xpath("//*//ol[@class='basic-search-results-lists expanded-view']//li[@class='compact']//span[@class='result-heading']//a/following-sibling::text()")
        Leg_Title = tree.xpath("//*//ol[@class='basic-search-results-lists expanded-view']//li[@class='compact']//span[@class='result-title bottom-padding']//text()")
        Date_Intro = tree.xpath("//div[@id='main']//li[@class='compact']//span[@class='result-item'][1]/span[1]/a[1]/following-sibling::text()[1]")
        House_or_Senate = ["House"]
        if len(Bill_Code) == len(Latest) == len(Congress) == len(Leg_Title) == len(Date_Intro):
            test = zip(Bill_Code,Latest,Congress,Leg_Title,Date_Intro,House_or_Senate*len(Date_Intro))
        else:
            print("Error!!!")
        test = list(test)
        results.append(test)
    results = pd.DataFrame(test, columns = ["Measure","Latest","Congress","Legislation Title","Date Introduced","House or Senate"]).replace(regex=["— "],value="")
    for x in range(len(results.iloc[:,4])):
        match = re.search(r'\d{2}/\d{2}/\d{4}', results.iloc[x,4])
        results.iloc[x,4] = datetime.strptime(match.group(), '%m/%d/%Y').date()    
    hold = []
    for x in range(len(results)):
        try:
            match = re.search(r'\d{2}/\d{2}/\d{4}', results.iloc[x,1])
            hold.append(datetime.strptime(match.group(), '%m/%d/%Y').date())
        except:
            hold.append(None)
    if len(hold) == len(results):
        results.insert(loc = 2,column="Latest Action Date",value=hold)
    results = results.replace(regex=["— "],value="")
    df = results
    Q1 = df[datetime.strptime(str("01/01/"+datestart.strftime("%Y")),"%m/%d/%Y").date()<df['Latest Action Date']]
    Q1 = Q1[datetime.strptime(str("03/31/"+datestart.strftime("%Y")),"%m/%d/%Y").date()>=Q1['Latest Action Date']]
    Q2 = df[datetime.strptime(str("03/31/"+datestart.strftime("%Y")),"%m/%d/%Y").date()<df['Date Introduced']]
    Q2 = Q2[datetime.strptime(str("06/30/"+datestart.strftime("%Y")),"%m/%d/%Y").date()>=Q2['Date Introduced']]
    Q3 = df[datetime.strptime(str("06/30/"+datestart.strftime("%Y")),"%m/%d/%Y").date()<df['Date Introduced']]
    Q3 = Q3[datetime.strptime(str("09/30/"+datestart.strftime("%Y")),"%m/%d/%Y").date()>=Q3['Date Introduced']]
    Q4 = df[datetime.strptime(str("09/30/"+datestart.strftime("%Y")),"%m/%d/%Y").date()<df['Date Introduced']]
    Q4 = Q4[datetime.strptime(str("12/31/"+datestart.strftime("%Y")),"%m/%d/%Y").date()>=Q4['Date Introduced']]

    l = {"Total Unreported Bills (House)":len(df),
        "Q1 Bills":len(Q1),
        "Q2 Bills":len(Q2),
        "Q3 Bills":len(Q3),
        "Q4 Bills":len(Q4)}
    summary = pd.DataFrame.from_dict(l,orient="index",columns=[("Unreported Bills Considered by the House between "+(datestart.strftime("%m/%d/%Y")+" and "+dateend.strftime("%m/%d/%Y")))])

    return(results,summary)

In [14]:
def unrep_senate(start = "01/01/2020",end=None):
    from lxml import html
    import requests
    import re
    import pandas as pd
    from datetime import datetime, timedelta
    if end == None:
        dateend = end = datetime.now().date()       
    else:
        dateend = datetime.strptime(end, '%m/%d/%Y').date()
    datestart = datetime.strptime(start, '%m/%d/%Y').date()
    webpage = requests.get("https://www.congress.gov/advanced-search/command-line?query=actionCode%3A16000+NOT+14000+latestActionDateStr%3A%5B%22"+str(datestart.strftime("%Y-%m-%d"))+"%22+TO+%22"+str(dateend.strftime("%Y-%m-%d"))+"%22%5D&searchResultViewType=compact&q=%7B%22type%22%3A%5B%22bills%22%2C%22joint-resolutions%22%5D%7D&pageSize=250")
    tree = html.fromstring(webpage.content)
    try:
        iterations = int(re.sub("[^0-9]", "", str(tree.xpath("//div[@class='pagination']//span[@class='results-number']/text()"))))
    except:
        iterations = 1
    results = []
    for i in range(int(iterations)):
        webpage = requests.get("https://www.congress.gov/advanced-search/command-line?query=actionCode%3A16000+NOT+14000+latestActionDateStr%3A%5B%22"+str(datestart.strftime("%Y-%m-%d"))+"%22+TO+%22"+str(dateend.strftime("%Y-%m-%d"))+"%22%5D&searchResultViewType=compact&q=%7B%22type%22%3A%5B%22bills%22%2C%22joint-resolutions%22%5D%7D&pageSize=250&page="+str(i))
        tree = html.fromstring(webpage.content)
        Bill_Code = tree.xpath("//*//ol[@class='basic-search-results-lists expanded-view']//li[@class='compact']/span[@class='result-heading']/a/text()")
        Latest = tree.xpath("//*//li[@class='compact']//span[@class='result-item'][./strong='Latest Action:']/span[1]/text()[1]")
        Congress = tree.xpath("//*//ol[@class='basic-search-results-lists expanded-view']//li[@class='compact']//span[@class='result-heading']//a/following-sibling::text()")
        Leg_Title = tree.xpath("//*//ol[@class='basic-search-results-lists expanded-view']//li[@class='compact']//span[@class='result-title bottom-padding']//text()")
        Date_Intro = tree.xpath("//div[@id='main']//li[@class='compact']//span[@class='result-item'][1]/span[1]/a[1]/following-sibling::text()[1]")
        House_or_Senate = ["Senate"]
        if len(Bill_Code) == len(Latest) == len(Congress) == len(Leg_Title) == len(Date_Intro):
            test = zip(Bill_Code,Latest,Congress,Leg_Title,Date_Intro,House_or_Senate*len(Date_Intro))
        else:
            print("Error!!!")
        test = list(test)
        results.append(test)
    results = pd.DataFrame(test, columns = ["Measure","Latest","Congress","Legislation Title","Date Introduced","House or Senate"]).replace(regex=["— "],value="")
    for x in range(len(results.iloc[:,4])):
        match = re.search(r'\d{2}/\d{2}/\d{4}', results.iloc[x,4])
        results.iloc[x,4] = datetime.strptime(match.group(), '%m/%d/%Y').date()
    hold = []
    for x in range(len(results)):
        try:
            match = re.search(r'\d{2}/\d{2}/\d{4}', results.iloc[x,1])
            hold.append(datetime.strptime(match.group(), '%m/%d/%Y').date())
        except:
            hold.append(None)
    if len(hold) == len(results):
        results.insert(loc = 2,column="Latest Action Date",value=hold)
    results = results.replace(regex=["— "],value="")
    df = results
    Q1 = df[datetime.strptime(str("01/01/"+datestart.strftime("%Y")),"%m/%d/%Y").date()<df['Latest Action Date']]
    Q1 = Q1[datetime.strptime(str("03/31/"+datestart.strftime("%Y")),"%m/%d/%Y").date()>=Q1['Latest Action Date']]
    Q2 = df[datetime.strptime(str("03/31/"+datestart.strftime("%Y")),"%m/%d/%Y").date()<df['Date Introduced']]
    Q2 = Q2[datetime.strptime(str("06/30/"+datestart.strftime("%Y")),"%m/%d/%Y").date()>=Q2['Date Introduced']]
    Q3 = df[datetime.strptime(str("06/30/"+datestart.strftime("%Y")),"%m/%d/%Y").date()<df['Date Introduced']]
    Q3 = Q3[datetime.strptime(str("09/30/"+datestart.strftime("%Y")),"%m/%d/%Y").date()>=Q3['Date Introduced']]
    Q4 = df[datetime.strptime(str("09/30/"+datestart.strftime("%Y")),"%m/%d/%Y").date()<df['Date Introduced']]
    Q4 = Q4[datetime.strptime(str("12/31/"+datestart.strftime("%Y")),"%m/%d/%Y").date()>=Q4['Date Introduced']]

    l = {"Total Unreported Bills (Senate)":len(df),
        "Q1 Bills":len(Q1),
        "Q2 Bills":len(Q2),
        "Q3 Bills":len(Q3),
        "Q4 Bills":len(Q4)}
    summary = pd.DataFrame.from_dict(l,orient="index",columns=[("Unreported Bills Considered by the Senate between "+(datestart.strftime("%m/%d/%Y")+" and "+dateend.strftime("%m/%d/%Y")))])

    return(results,summary)

In [15]:
def amendments_senate(start = "01/01/2020",end=None):
    from lxml import html
    import requests
    import re
    import pandas as pd
    from datetime import datetime, timedelta
    if end == None:
        dateend = end = datetime.now().date()       
    else:
        dateend = datetime.strptime(end, '%m/%d/%Y').date()
    datestart = datetime.strptime(start, '%m/%d/%Y').date()
    webpage = requests.get("https://www.congress.gov/advanced-search/command-line?query=actionCode:(94000%20OR%2095000)%20latestActionDateStr:[%22"+str(datestart.strftime("%Y-%m-%d"))+"%22%20TO%20%22"+str(dateend.strftime("%Y-%m-%d"))+"%22]%20billType:%22SAmdt%22&searchResultViewType=compact&KWICView=false&pageSize=250")
    tree = html.fromstring(webpage.content)
    try:
        iterations = int(re.sub("[^0-9]", "", str(tree.xpath("//div[@class='pagination']//span[@class='results-number']/text()"))))
    except:
        iterations = 1
    results = []
    
    for i in range(int(iterations)):
        webpage = requests.get("https://www.congress.gov/advanced-search/command-line?query=actionCode:(94000%20OR%2095000)%20latestActionDateStr:[%22"+str(datestart.strftime("%Y-%m-%d"))+"%22%20TO%20%22"+str(dateend.strftime("%Y-%m-%d"))+"%22]%20billType:%22SAmdt%22&searchResultViewType=compact&KWICView=false&pageSize=250&page="+str(i))
        tree = html.fromstring(webpage.content)
        Amdt_No = tree.xpath("//*//li[@class='compact']//span[@class='result-heading amendment-heading']/a[1]/text()[1]")
        Amends = []
        hold = tree.xpath("//*//li[@class='compact']//span[@class='result-heading amendment-heading']//text()")
        for i in range(0,len(hold)):
            if "to" not in hold:
                if "Congress" in hold[i]:
                    Amends.append(None)
                else:
                    continue
            else:
                if "to" in hold[i]:
                    Amends.append(hold[i+1])
                elif "Congress" in hold[i] and "to" not in hold[i-2]:
                    Amends.append(None)
                elif i == len(hold):
                    Amends.append(None)
                else:
                    continue
        l = tree.xpath("//*//li[@class='compact']//span[@class='result-item'][contains(.//strong,'Amends')]//span/a/text()[normalize-space()]")
        for i in range(len(Amends)):
            if Amends[i] == None or Amends[i] == []:
                Amends[i] = l[i]
        Sponsor = tree.xpath("//*//li[@class='compact']//span[@class='result-item'][contains(.//strong,'Sponsor')]//span/a/text()[normalize-space()]")
        Party = [[]]*len(Sponsor)
        for i in range(len(Sponsor)):
            Party[i] = str(Sponsor[i])[int(str(Sponsor[i]).find("["))+1:int(str(Sponsor[i]).find("["))+2].strip()
        for i in range(len(Sponsor)):
            Sponsor[i] = str(Sponsor[i])[:int(str(Sponsor[i]).find("["))].strip()
        new = []*2
        i = 0
        Latest = tree.xpath("//*//li[@class='compact']//span[@class='result-item'][contains(.//strong,'Latest')]//text()[normalize-space()]")
        for r in range(len(Latest)):
            if r == 0:
                if "Latest Action" not in Latest[r]:
                    new[i] = str(new[i]) + str(Latest[r])
                new.append([])
            elif "Latest Action" not in Latest[r]:
                new[i] = str(new[i]) + str(Latest[r])
            else:
                i += 1
                new.append([])
                continue
        Latest = new
        Outcome = [[]]*int(len(Latest))
        x = 0
        for i in Latest:
            if "not agreed" in i:
                Outcome[x] = "NA"
                x += 1
            else:
                Outcome[x] = "A"
                x += 1
        Action_Type = [[]]*int(len(Latest))
        x = 0
        for i in Latest:
            if "Unanimous" in i or "unanimous" in i:
                Action_Type[x] = "UC"
                x += 1
            elif "Yea-Nay" in i:
                Action_Type[x] = "RCV"
                x += 1
            elif "Voice Vote" in i:
                Action_Type[x] = "VV"
                x += 1
            else:
                Action_Type[x] = str(i) + "ERROR"
                x += 1   
        if len(Amdt_No) == len(Amends) == len(Sponsor) == len(Latest) == len(Party) == len(Outcome) == len(Action_Type):
                test = zip(Amdt_No,Amends,Sponsor,Party,Latest,Outcome,Action_Type)
                test = list(test)
                results.append(test)
        else:
                print("Error!!!")
                print(len(Amdt_No),len(Amends),len(Sponsor),len(Party),len(Latest),len(Outcome),len(Action_Type))


    results = pd.DataFrame(test, columns = ["S.Amdt.No","Amends","Sponsor","Sponsor Party","Date of Action","Outcome","Action Type"])
    df = results
    l=[len(df),len(df[df["Sponsor Party"]=="R"]),len(df[df["Sponsor Party"]=="D"]),len(df[df["Action Type"]=="RCV"]),len(df[df["Action Type"]=="VV"]),len(df[df["Action Type"]=="UC"])]
    l_labels = ["Total Amendments","R Amdts","D Amdts","RCV","VV","UC"]
    ltype = [len(df[df["Amends"].str.contains("H.R.")]) + len(df[df["Amends"].str.contains("Treaty")]) + len(df[~df["Amends"].str.contains("S.J")][~df["Amends"].str.contains("S.Res")][df["Amends"].str.contains("S.")]),
             len(df[df["Amends"].str.contains("Res")]),
             len(df[df["Amends"].str.contains("Amdt")]),
             len(df[df["Amends"].str.contains("con")])]
    ltype_labels = ["Bills Amended","Resolutions Amended","Amdts Amended","Budget Resolutions"]

    summary = pd.DataFrame(ltype,index=ltype_labels,columns=["Summary Data for Senate Amendments"])
    summary = summary.append(pd.DataFrame(l,index=l_labels,columns=["Summary Data for Senate Amendments"]))
    summary = summary.append(pd.DataFrame([(datestart.strftime("%m/%d/%Y")+" through "+dateend.strftime("%m/%d/%Y"))],index=["Date Range"],columns=["Summary Data for Senate Amendments"]))    

    return(results,summary)

In [17]:
def cloture(start = "01/01/2020",end=None):
    from lxml import html
    import requests
    import re
    import pandas as pd
    from datetime import datetime, timedelta
    if end == None:
        dateend = end = datetime.now().date()       
    else:
        dateend = datetime.strptime(end, '%m/%d/%Y').date()
    datestart = datetime.strptime(start, '%m/%d/%Y').date()
    i = 116
    yr = int(datestart.strftime("%Y"))
    if yr % 2 == 0:
        yr = yr - 1
    urlstr = ("https://www.senate.gov/legislative/cloture/" + str(i) + ".htm")
    webpage = requests.get(urlstr)
    tree = html.fromstring(webpage.content)
    daterange = str(tree.xpath("//table/tr/td[@class='contenttext']/text()"))
    while str(yr) not in daterange:
        i = i - 1
        urlstr = ("https://www.senate.gov/legislative/cloture/" + str(i) + ".htm")
        webpage = requests.get(urlstr)
        tree = html.fromstring(webpage.content)
        daterange = str(tree.xpath("//table/tr/td[@class='contenttext']/text()"))
    l = list(tree.xpath("//table//tr//td[@valign='top'][3]/text()[normalize-space()]"))
    l_topop = list()
    for row in range(len(l)):
        if "\xa0" in l[row]:
            l[row-1] = l[row-1] + str(re.sub("\xa0\xa0\xa0", "", l[row]))
            l_topop.append(row)
    l = pd.DataFrame(l)
    l = l.drop(l_topop)
    l = l.values.tolist()
    lx = list()
    for i in l:
        v = re.sub("\[","",str(i))
        v = re.sub("\]","",str(v))
        v = re.sub("'","",str(v))
        lx.append(v)

    l = lx
    hold = list(tree.xpath("//table//tr//td[@valign='top'][6]/text()[normalize-space()]"))
    new = list()
    for i in hold:
        i = re.sub("-\n", "-",str(i))
        new.append(re.sub("[^0-9,-]", "",i))
    dates = list()
    for x in range(1,len(new)):
        y = tree.xpath(("//table//tr[./td[@valign='top']]["+str(x)+"]//td[@valign='top'][5]/text()[normalize-space()]"))
        if len(y) > 0:
             dates.append(y)
        else:
            dates.append(None)

    for row in range(len(dates)):
        if dates[row] is not None:
            if len(dates[row]) > 1:
                dates.append(dates[row][1])
                del dates[row][1]  
    date_filed = tree.xpath("//table//tr//td[@valign='top'][1]/text()[normalize-space()]")
    measure = tree.xpath("//table//tr//td[@valign='top'][2]/a/text()[normalize-space()]")
    filer = tree.xpath("//table//tr//td[@valign='top'][4]/text()[normalize-space()]")
    res = tree.xpath("//table//tr//td[@valign='top'][7]/text()[normalize-space()]")
    double_list = list()
    for i in range(len(hold)):
        bool_ = list()
        for char in  hold[i]:
            bool_.append("true") if char.isdigit() else bool_.append("false")
        if "true" in bool_ and "\xa0" in str(hold[i]):
            double_list.append(i)
    for i in range(len(double_list)):
        double_list[i] = int(re.sub("[^0-9]", "",str(double_list[i])))
        v = double_list[i] - 1
        date_filed.append(date_filed[v])
        measure.append(measure[v])
        l.append(l[v])
        filer.append(filer[v])

    test = zip(date_filed,measure,l,filer,dates,new,res)
    test = list(test)
    results = pd.DataFrame(test,columns = ["Date Introduced","Measure","Subject","Filed By","Date","Vote","Result"])
    leg_type = []
    for row in results.iloc[:,[1,2]].index:
        if "motion to proceed" in str(results.iloc[row,2]):
            leg_type.append("Motion to Proceed")
        elif "PN" in results.iloc[row,1]:
            leg_type.append("Nomination")
        elif "PN" not in results.iloc[row,1] and "motion to proceed" not in results.iloc[row,2]:
            leg_type.append("Legislation")
        else:
            leg_type.append("ERROR")
    results.insert(loc=3,column="Legislation Type",value=leg_type)
    z = str(tree.xpath("//table//tr//td[@class='contenttext']/text()[normalize-space()]"))
    yr = int(re.search(r'\d{4}',z).group())
    for x in range(len(results.iloc[:,5])):
        datest = re.sub("\\[","",str(results.iloc[x,5]))
        datest = re.sub("\\]","",datest)
        datest = re.sub("Sept","Sep",datest)
        datest = re.sub("July","Jul",datest)
        datest = re.sub("June","Jun",datest)
        datest = re.sub("'","",datest)
        datest = datest.strip()
        if len(str(results.iloc[x,5])) > 2 and results.iloc[x,5] is not None:
            try:
                results.iloc[x,5] = datetime.strptime(datest, ('%b %d')).date()
            except:
                results.iloc[x,5] = datetime.strptime(datest, ('%Y-%m-%d')).date()
    res = list()
    for x in range(len(results)):
        try:
            if str(type(results.iloc[:,5][x])) == "<class 'datetime.date'>" and results.iloc[:,5][x] >= results.iloc[:,5][x+1]:
                res.append("True")
            elif str(type(results.iloc[:,5][x])) == "<class 'datetime.date'>":
                res.append("False")
            else:
                res.append("")
        except:
            res.append("")
    stop_val = res.index("False")

    for row in range(len(results.iloc[:,5])):
        if row <=stop_val:
            try:
                results.iloc[:,5][row] = results.iloc[:,5][row].replace(year=yr+1)
            except:
                continue
        elif row > stop_val:
            try:
                results.iloc[:,5][row] = results.iloc[:,5][row].replace(year=yr)
            except:
                continue

    results = results[(results['Date'] > datestart) & (results['Date'] < dateend)]
    df = results
    Q1 = df[datetime.strptime(str("01/01/"+datestart.strftime("%Y")),"%m/%d/%Y").date()<df['Date']]
    Q1 = Q1[datetime.strptime(str("03/31/"+datestart.strftime("%Y")),"%m/%d/%Y").date()>=Q1['Date']]
    Q2 = df[datetime.strptime(str("03/31/"+datestart.strftime("%Y")),"%m/%d/%Y").date()<df['Date']]
    Q2 = Q2[datetime.strptime(str("06/30/"+datestart.strftime("%Y")),"%m/%d/%Y").date()>=Q2['Date']]
    Q3 = df[datetime.strptime(str("06/30/"+datestart.strftime("%Y")),"%m/%d/%Y").date()<df['Date']]
    Q3 = Q3[datetime.strptime(str("09/30/"+datestart.strftime("%Y")),"%m/%d/%Y").date()>=Q3['Date']]
    Q4 = df[datetime.strptime(str("09/30/"+datestart.strftime("%Y")),"%m/%d/%Y").date()<df['Date']]
    Q4 = Q4[datetime.strptime(str("12/31/"+datestart.strftime("%Y")),"%m/%d/%Y").date()>=Q4['Date']]
    ###Legislation
    x = Q1,Q2,Q3,Q4
    I = list()
    F = list()
    N = list()
    V = list()
    Re = list()
    W = list()
    for i in x:
        I.append(len(i[~i["Measure"].str.contains("PN")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="I"]))
        F.append(len(i[~i["Measure"].str.contains("PN")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="F"]))
        N.append(len(i[~i["Measure"].str.contains("PN")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="N"]))
        V.append(len(i[~i["Measure"].str.contains("PN")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="V"]))
        Re.append(len(i[~i["Measure"].str.contains("PN")][i["Subject"].str.contains("reconsidered")]))
        W.append(len(i[~i["Measure"].str.contains("PN")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="W"]))

    z = list(zip(I,F,N,V,Re,W))
    legislation = pd.DataFrame(z,columns=["I","F","N","V","Recons","W"],index=[str("Q1-"+dateend.strftime("%Y")),str("Q2-"+dateend.strftime("%Y")),str("Q3-"+dateend.strftime("%Y")),str("Q4-"+dateend.strftime("%Y"))])
    ###MTP
    I = list()
    F = list()
    N = list()
    V = list()
    Re = list()
    W = list()
    for i in x:
        I.append(len(i[~i["Measure"].str.contains("PN")][i["Subject"].str.contains("motion to proceed")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="I"]))
        F.append(len(i[~i["Measure"].str.contains("PN")][i["Subject"].str.contains("motion to proceed")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="F"]))
        N.append(len(i[~i["Measure"].str.contains("PN")][i["Subject"].str.contains("motion to proceed")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="N"]))
        V.append(len(i[~i["Measure"].str.contains("PN")][i["Subject"].str.contains("motion to proceed")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="V"]))
        Re.append(len(i[~i["Measure"].str.contains("PN")][i["Subject"].str.contains("motion to proceed")][i["Subject"].str.contains("reconsidered")]))
        W.append(len(i[~i["Measure"].str.contains("PN")][i["Subject"].str.contains("motion to proceed")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="W"]))

    z = list(zip(I,F,N,V,Re,W))
    mtp = pd.DataFrame(z,columns=["I","F","N","V","Recons","W"],index=[str("Q1-"+dateend.strftime("%Y")),str("Q2-"+dateend.strftime("%Y")),str("Q3-"+dateend.strftime("%Y")),str("Q4-"+dateend.strftime("%Y"))])
    ###NOMS
    x = Q1,Q2,Q3,Q4
    I = list()
    F = list()
    N = list()
    V = list()
    Re = list()
    W = list()
    for i in x:
        I.append(len(i[i["Measure"].str.contains("PN")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="I"]))
        F.append(len(i[i["Measure"].str.contains("PN")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="F"]))
        N.append(len(i[i["Measure"].str.contains("PN")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="N"]))
        V.append(len(i[i["Measure"].str.contains("PN")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="V"]))
        Re.append(len(i[i["Measure"].str.contains("PN")][i["Subject"].str.contains("reconsidered")]))
        W.append(len(i[i["Measure"].str.contains("PN")][~i["Subject"].str.contains("reconsidered")][i["Result"]=="W"]))

    z = list(zip(I,F,N,V,Re,W))
    noms = pd.DataFrame(z,columns=["I","F","N","V","Recons","W"],index=[str("Q1-"+dateend.strftime("%Y")),str("Q2-"+dateend.strftime("%Y")),str("Q3-"+dateend.strftime("%Y")),str("Q4-"+dateend.strftime("%Y"))])

    noms.columns = pd.MultiIndex.from_product([noms.columns, ['Nominations']])
    legislation.columns = pd.MultiIndex.from_product([legislation.columns, ['Legislation']])
    mtp.columns = pd.MultiIndex.from_product([mtp.columns, ['Motion to Proceed']])
    legislation = legislation.merge(mtp,left_index=True,right_index=True)
    summary = legislation.merge(noms,left_index=True,right_index=True)

    return(results,summary)

In [18]:
def workingdays(start="01/01/2020",end=None):
    from lxml import html
    import requests
    import re
    import pandas as pd
    from datetime import datetime, timedelta
    def timeconv(lst):
        x = str(lst[0])
        x = re.sub("a.m.","AM",x)
        x = re.sub("p.m.","PM",x)
        x = re.sub("noon","PM",x)
        x = re.sub("\[","",x)
        x = re.sub("\]","",x)
        x = re.sub("\'","",x)
        x = re.sub(",","",x)
        try:
            val = datetime.strptime(x,"%I %p").time()
            lst[0] = val
            return(val)

        except:
            val = datetime.strptime(x,"%I:%M %p").time()
            lst[0] = val
            return(val)
    if end == None:
        dateend = end = datetime.now().date()       
    else:
        dateend = datetime.strptime(end, '%m/%d/%Y').date()
    datestart = datetime.strptime(start, '%m/%d/%Y').date()
    class TestClass:
        def __init__(self,x,y):
            self.datestart = x
            self.url = y
    def web_test(datestart):
        from lxml import html
        import requests
        url = "https://www.congress.gov/congressional-record/"+str('{d.year}/{d.month}/{d.day}'.format(d=datestart))+"/daily-digest"
        while ('404' or '200') in str(requests.get(url)):
            datestart = datestart + timedelta(days=1)
            url = "https://www.congress.gov/congressional-record/"+str('{d.year}/{d.month}/{d.day}'.format(d=datestart))+"/daily-digest"
            if datestart >= dateend:
                break
        return(TestClass(datestart,url))

    conv_ = "convened at"
    adj_ = "adjourned at"
    diff = (dateend - datestart).days
    df = pd.DataFrame(columns=["House or Senate","Date","Time Convened","Time Adjourned","Working Day?"])

    for y in range(diff):
        datestart1 = datestart + (timedelta(days=1)*0 if y == 0 else timedelta(days=1)*1)
        url, datestart = web_test(datestart1).url, web_test(datestart1).datestart
        if datestart <= dateend:
            l = ["Senate","House"]
            webpage = requests.get(url)
            tree = html.fromstring(webpage.content) 
            for i in l:
                date = list()
                conv = list()
                adj = list()
                insession = list()
                house_or_senate = list()
                strg = str(tree.xpath("//*//div[@class='main-wrapper'][./center/h2='Senate']//p[strong='Adjournment:']//text()[1]"))
                strg= re.sub(",","",strg)
                strg = re.sub("\"","",strg)
                strg = re.sub("\'","",strg)
                strg = re.sub("\[","",strg)
                strg = re.sub("\]","",strg)
                date.append(datestart.strftime("%m/%d/%Y"))
                if "not in session" in strg:
                    conv.append(None)
                    adj.append(None)
                    insession.append("NS")
                    house_or_senate.append(i)
                elif i not in strg:
                    house_or_senate.append(i)
                    strg = str(tree.xpath("//div[@class='main-wrapper']/p[1]/text()"))
                    strg= re.sub(",","",strg)
                    strg = re.sub("\"","",strg)
                    strg = re.sub("\'","",strg)
                    strg = re.sub("\[","",strg)
                    strg = re.sub("\]","",strg)
                    if "not in session" in strg:
                        conv.append(None)
                        adj.append(None)
                        insession.append("NS")
                    if "Senate" in i:
                        strg = strg[:strg.find("House")]
                        if "pro forma" in strg:
                            conv.append(None)
                            adj.append(None)
                            insession.append("pf")
                    else:
                        strg = strg[strg.find("House"):]
                        if "pro forma" in strg:
                            conv.append(None)
                            adj.append(None)
                            insession.append("pf")                    
                elif "Senate" in i:
                    try:
                        conv_ = "convened at"
                        house_or_senate.append(i)
                        strg = strg[:strg.find("House")]
                        conv.append(strg[strg.find(conv_)+len(conv_):strg.find(conv_)+(strg[strg.find(conv_):]).find("and")].strip())
                        adj.append(strg[strg.find(adj_)+len(adj_):strg.find(adj_)+(strg[strg.find(adj_):]).find("until")].strip())
                        timein = (datetime.combine(datetime(1,1,1,0,0,0), timeconv(adj))-(datetime.combine(datetime(1,1,1,0,0,0), timeconv(conv)))).seconds//60
                        if timein >= 60:
                            insession.append("x")
                        else:
                            insession.append("NS")    
                    except:
                        try:
                            conv_ = "convened at"
                            adj = list()
                            conv = list()
                            house_or_senate.append(i)
                            strg = strg[:strg.find("House")]
                            conv.append(strg[strg.find(conv_)+len(conv_):strg.find(conv_)+len(conv_)+strg[strg.find(conv_):].find("on")+len(conv_)].strip())
                            adj.append(strg[strg.find(adj_)+len(adj_):strg.find(adj_)+(strg[strg.find(adj_):]).find("on")].strip())
                            timein = (datetime.combine(datetime(1,1,1,0,0,0), timeconv(adj))-(datetime.combine(datetime(1,1,1,0,0,0), timeconv(conv)))).seconds//60
                            if timein >= 60:
                                insession.append("x")
                            else:
                                insession.append("NS")                            
                        except:
                            try:
                                conv_ = "convened at"
                                adj = list()
                                conv = list()
                                house_or_senate.append(i)
                                strg = strg[:strg.find("House")]
                                conv.append(strg[strg.find(conv_)+len(conv_):strg.find(conv_)+len(conv_)+strg[strg.find(conv_):].find("and")+len(conv_)].strip())
                                adj.append(strg[strg.find(adj_)+len(adj_):strg.find(adj_)+(strg[strg.find(adj_):]).find("on")].strip())
                                timein = (datetime.combine(datetime(1,1,1,0,0,0), timeconv(adj))-(datetime.combine(datetime(1,1,1,0,0,0), timeconv(conv)))).seconds//60
                                if timein >= 60:
                                    insession.append("x")
                                else:
                                    insession.append("NS") 
                            except:
                                insession.append("INPUT BY HAND")
                                house_or_senate.append(i)
                                conv.append(str("Error: "+strg))
                                adj.append(str("Error: "+strg))
                elif "House" in i:
                    try:
                        house_or_senate.append(i)
                        strg = strg[strg.find("House"):]
                        conv_ = "met at"
                        conv.append(strg[strg.find(conv_)+len(conv_):strg.find(conv_)+(strg[strg.find(conv_):]).find("and")].strip())
                        adj.append(strg[strg.find(adj_)+len(adj_):].strip())
                        timein = (datetime.combine(datetime(1,1,1,0,0,0), timeconv(adj))-(datetime.combine(datetime(1,1,1,0,0,0), timeconv(conv)))).seconds//60
                        if timein >= 60:
                            insession.append("x")
                        else:
                            insession.append("pf")
                    except:
                        insession.append("INPUT BY HAND")
                        house_or_senate.append(i)
                        conv.append(str("Error: "+strg))
                        adj.append(str("Error: "+strg))
                df = df.append(pd.DataFrame(zip(house_or_senate,date,conv,adj,insession),columns=["House or Senate","Date","Time Convened","Time Adjourned","Working Day?"]))
        else:
            break
    results = df
    dfhouse = df
    df = df[df["House or Senate"]=="Senate"]
    Q1 = df[str("01/01/"+datestart.strftime("%Y"))<df['Date']]
    Q1 = Q1[str("03/31/"+datestart.strftime("%Y"))>=Q1['Date']]
    Q2 = df[str("03/31/"+datestart.strftime("%Y"))<df['Date']]
    Q2 = Q2[str("06/30/"+datestart.strftime("%Y"))>=Q2['Date']]
    Q3 = df[str("06/30/"+datestart.strftime("%Y"))<df['Date']]
    Q3 = Q3[str("09/30/"+datestart.strftime("%Y"))>=Q3['Date']]
    Q4 = df[str("09/30/"+datestart.strftime("%Y"))<df['Date']]
    Q4 = Q4[str("12/31/"+datestart.strftime("%Y"))>=Q4['Date']]
    ls = [len(Q1[Q1["Working Day?"] == "x"]),len(Q2[Q2["Working Day?"] == "x"]),len(Q3[Q3["Working Day?"] == "x"]),len(Q4[Q4["Working Day?"] == "x"])]

    dfhouse = dfhouse[dfhouse["House or Senate"]=="House"]
    Q1 = dfhouse[str("01/01/"+datestart.strftime("%Y"))<dfhouse['Date']]
    Q1 = Q1[str("03/31/"+datestart.strftime("%Y"))>=Q1['Date']]
    Q2 = dfhouse[str("03/31/"+datestart.strftime("%Y"))<dfhouse['Date']]
    Q2 = Q2[str("06/30/"+datestart.strftime("%Y"))>=Q2['Date']]
    Q3 = dfhouse[str("06/30/"+datestart.strftime("%Y"))<dfhouse['Date']]
    Q3 = Q3[str("09/30/"+datestart.strftime("%Y"))>=Q3['Date']]
    Q4 = dfhouse[str("09/30/"+datestart.strftime("%Y"))<dfhouse['Date']]
    Q4 = Q4[str("12/31/"+datestart.strftime("%Y"))>=Q4['Date']]
    lsH = [len(Q1[Q1["Working Day?"] == "x"]),len(Q2[Q2["Working Day?"] == "x"]),len(Q3[Q3["Working Day?"] == "x"]),len(Q4[Q4["Working Day?"] == "x"])]
    if len(ls) < len(lsH):
        for i in range(len(lsH)-len(ls)):
            ls.append(None)
    elif len(lsH) < len(ls):
        for i in range(len(ls)-len(lsH)):
            lsH.append(None)
    summary = pd.DataFrame(zip(ls,lsH),index=["Q1","Q2","Q3","Q4"],columns=["Working Days - Senate","Working Days - House"]) 
    l = pd.DataFrame([[start,dateend.strftime("%m/%d/%Y")]],columns=["Working Days - Senate","Working Days - House"],index=["Date Range"])
    summary = summary.append(l)

    return(results,summary)

In [98]:
from PyQt5.QtCore import QDateTime, Qt, QTimer,pyqtSlot,QUrl,QDir
from PyQt5.QtWidgets import (QApplication, QCheckBox, QComboBox, QDateTimeEdit,
        QDial, QDialog, QGridLayout, QGroupBox, QHBoxLayout, QLabel, QLineEdit,
        QProgressBar, QPushButton, QRadioButton, QScrollBar, QSizePolicy,
        QSlider, QSpinBox, QStyleFactory, QTableWidget, QTabWidget, QTextEdit,
        QVBoxLayout, QWidget,QMessageBox,QApplication, QWidget, QInputDialog, QLineEdit, QFileDialog)
from PyQt5.QtGui import QIcon,QValidator
def getall_summary(start,end):
    import pandas as pd
    from datetime import datetime
    import re
    out = re.sub("/","\\\\",(QDir.homePath()+"/Downloads/"+'HCISummary'+datetime.now().strftime("-%m-%Y")+'.xlsx'))
    writer = pd.ExcelWriter(out, engine='xlsxwriter')    
    UNresults,UNsummary = unrep_house(start,end)
    UNSresults,UNSsummary = unrep_senate(start,end)
    AMresults,AMsummary = amendments_senate(start,end)
    CLresults,CLsummary = cloture(start,end)
    WDresults,WDsummary = workingdays(start,end)
    
    dfs = {'Conference': conference(start,end), 
           'House Unreported': UNsummary, 
           'Senate Unreported': UNSsummary,
          'Senate Amendments': AMsummary,
           'Cloture':CLsummary, 
           'Working Days':WDsummary,
          'Reported Bills':reportedbills(start,end)}
    for sheetname, df in dfs.items():  # loop through `dict` of dataframes
        df.to_excel(writer, sheet_name=sheetname)  # send df to writer
        worksheet = writer.sheets[sheetname]  # pull worksheet object
        for idx, col in enumerate(df):  # loop through all columns
            nlevels = df.index.nlevels 
            series = df[col]
            max_len = max((
                series.astype(str).map(len).max(),  # len of largest item
                len(str(series.name))  # len of column name/header
                )) + 1  # adding a little extra space    
            worksheet.set_column(idx+nlevels, idx+nlevels, max_len)  # set column width
    writer.sheets['Conference'].set_column('A:A',1)
    writer.sheets['House Unreported'].set_column('A:A',30)
    writer.sheets['Senate Unreported'].set_column('A:A',30)
    writer.sheets['Senate Amendments'].set_column('A:A',30)
    writer.sheets['Cloture'].set_column('A:A',5)
    writer.sheets['Working Days'].set_column('A:A',5)
    writer.save()
    out = re.sub("/","\\\\",(QDir.homePath()+"/Downloads/"+'HCIRaw'+datetime.now().strftime("-%m-%Y")+'.xlsx'))
    writer = pd.ExcelWriter(out, engine='xlsxwriter')
   
    dfs = {'Conference': conference(start,end), 
           'House Unreported': UNresults, 
           'Senate Unreported': UNSresults,
          'Senate Amendments': AMresults,
           'Cloture': CLresults, 
           'Working Days': WDresults,
          'Reported Bills':reportedbills(start,end)}
    for sheetname, df in dfs.items():  # loop through `dict` of dataframes
        df.to_excel(writer, sheet_name=sheetname)  # send df to writer
        worksheet = writer.sheets[sheetname]  # pull worksheet object
        for idx, col in enumerate(df):  # loop through all columns
            nlevels = df.index.nlevels 
            series = df[col]
            max_len = max((
                series.astype(str).map(len).max(),  # len of largest item
                len(str(series.name))  # len of column name/header
                )) + 1  # adding a little extra space   
            if max_len == 1:
                max_len = 10
            worksheet.set_column(idx+nlevels, idx+nlevels, max_len)  # set column width
    writer.save()

In [ ]:
# import base64
# import pandas as pd
# from IPython.display import HTML

# def create_download_link(df, title = "Download CSV file", filename = "data.csv"):
#     csv = df.to_csv()
#     b64 = base64.b64encode(csv.encode())
#     payload = b64.decode()
#     html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
#     html = html.format(payload=payload,title=title,filename=filename)
#     return HTML(html)